# GCP의 Storage와 BigQuery에 접근하기

___
## GCS (Google Cloud Storage)

- gsutil은 Cloud Shell에서 별도의 설정 없이 사용 가능
- Console에서도 파일 관리가 가능하나, 여기에서는 sh 사용법 위주로 설명

### gsutil cp (파일복사)

- 데이터를 업로드/복사할 때에는 `cp` 명령어를 사용

``` 
$ gsutil cp <원본주소> <사본주소>
$ gsutil cp sample.txt gs://ml_with_gcp/
```
- `gs://...`는 파일 경로를 나타내며 `gs://<버킷명>/<디렉터리명>/<파일명>`으로 구성
- 와일드카드`*`도 사용 가능하며 아래 사용 예를 참조:

```
# google storage는 gs://...로 시작
$ gsutil cp *.txt gs://...
```

- 디렉터리를 통째로 복사할 때는 `-r` 옵션을, 병렬 처리시에는 `-m`옵션을 사용 가능:

```
$ gsutil cp -r <원본주소> <사본주소>
$ gsutil -m cp -r <원본주소> <사본주소>
```

### gsutil rm (파일제거)

- `gsutil cp`와 유사한 방식으로 사용

```
$ gsutil rm <지울주소>
```

- 폴더 단위로 삭제`*`, 하위계층까지 삭제`**` 가능:

```
$ gsutil rm <지울주소>/*
$ gsutil rm <지울주소>/**
```

-`-r` 옵션을 사용 시 버전 이력까지 함께 제거

```
$ gsutil rm -r <지울주소>
```

### gstuil mv (파일이동)

- 파일을 이동하거나 파일의 이름을 변경할 때 사용
- 디렉터리 내 파일을 모두 옮기려면 다음과 같이 명령

```
$ gsutil mv <목표주소>/* <신규주소>
```

- 디렉터리 이름을 변경하려면 다음과 같이 명령

```
$ gsutil mv <과거주소> <신규주소>
```

※ GCS 에서는 파일 이동중에도 조작 가능

### 공개 버킷에서 데이터 복사

- GCS에 저장한 데이터들은 공개 상태로 설정 가능
- Google이 제공하는 '랜드셋 인공위성' 사진데이터는 공개되어 있고, 이를 복사하는 것도 가능

```
$ gsutil -m cp 
> gs://gcp-public-data-landsat/LT05/PRE/108/035/LT51080351995058HAJ00/*.TIF
> gs://<버킷이름>
```

- 명령어를 실행하면 총 87개의 TIF 파일을 복사

___
## BigQuery

- BigQuery는 빅데이터 분석 전용 데이터 웨어하우스임
- GCP 서비스 대부분은 BigQuery 관련 인터페이스를 갖추고 있어 머신러닝 활용에 용이

### 데이터 준비

- git을 통해 퍼블릭 데이터 접근

```
$ git clone https://github.com/hayatoy/bqsamples.git
```

- 위 데이터는 미국 사회보장국에서 제공하는 데이터를 기반으로, 2016년에 태어난 아이들의 이름 데이터임
- `names.csv`에는 데이터가, `schema.json`에는 스키마가 들어 있음

```
$ head bqsamples/names.csv
WY,F,2016,Hazel,10
WY,F,2016,Jasmine,5
WY,M,2016,Xander,7
AK,F,2016,Charlotte,34
AK,F,2016,Naomi,12
AK,F,2016,Ellie,10
AK,F,2016,Audriana,5
AK,M,2016,Owen,40
AK,M,2016,Mason,39
AK,M,2016,Dominic,11
```

- 헤더가 없는 쉼표로 구분된 텍스트 데이터
- 'state', 'gender' ,' year', 'name', 'count' 형태로 차례대로 적혀 있음
- 아래는 `schema.json` 데이터를 호출한 결과임

```
$ head bqsamples/schema.json
[
        {"name": "state", "type": "string", "mode": "nullable"},
        {"name": "gender", "type": "string", "mode": "nullable"},
        {"name": "year", "type": "integer", "mode": "nullable"},
        {"name": "name", "type": "string", "mode": "nullable"},
        {"name": "count", "type": "integer", "mode": "nullable"}
]```

### 데이터 세트 만들기

- BigQuery에서 데이터 세트를 만들 때는 다음 명령어를 실행

```
$ bq mk <데이터세트>
```

- `bq load` 명령어의 경우 데이터 업로드부터 테이블 생성까지 한번에 수행 가능

```
$ bq load <데이터세트>.<테이블> <원본데이터> <원본스키마>
```
- 스키마는 JSON 파일로 지정하는 방법 외 `<칼럼이름>:<데이터자료형>`의 형태로 지정할 수도 있음 (실수방지를 위해 권장하지 않음)

- shell 에서 쿼리를 실행하는 방법은 `bq query` 이후 쿼리를 작성한는 방식으로 가능

```
$ bq query "select name, count
> from kidnames2016.names
> where state = 'CA' and gender = 'F'
> order by count desc
> limit 5"

Waiting on bqjob_r70a28b20adc7d2f7_000001664152daa7_1 ... (0s) Current status: DONE   
+----------+-------+
|   name   | count |
+----------+-------+
| Mia      |  2785 |
| Sophia   |  2747 |
| Emma     |  2592 |
| Olivia   |  2533 |
| Isabella |  2350 |
+----------+-------+
```

- BigQuery의 경우 데이터 처리 속도가 빠르다 보니 너무 많이 사용하는 경우가 있음
- 따라서 칼럼을 제한하거나, 작은 테이블로 분할하는 작업을 선행하는 것도 좋은 방안임
- 쿼리 실행 전에 스캔 용량을 확인 해 보는 것도 좋은 방안임

```
$ bq query --dry_run "select name, count
from kidnames2016.names
where state = 'CA' and gender = 'F'
order by count desc
limit 5"

Query successfully validated. Assuming the tables are not modified, running this query will process 2173371 bytes of data.
```